In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

Load the data 

In [ ]:
df=pd.read_csv('./Data_project/bengaluru_house_prices.csv')
df.head()

In [ ]:
df.shape

Check the columns one by one

In [ ]:
df['area_type'].value_counts()

In [ ]:
df.groupby('area_type')['area_type'].agg('count')

Drop the unimportant columns

In [ ]:
df2=df.drop(['area_type','availability','society','balcony'],axis=1)
df2.head()

# Data Cleaning

In [ ]:
# Hundling missing value
df2.isnull().sum()

In [ ]:
# Fill the bath null values by median 

In [ ]:
df2['bath'].median()

In [ ]:
import math
bath_median=math.floor(df2['bath'].median())

In [ ]:
df2['bath']=df2['bath'].fillna(bath_median)

In [ ]:
df2.isnull().sum()

Drop the remain null values

In [ ]:
df3=df2.dropna()

In [ ]:
df3.isnull().sum()

In [ ]:
df3['size'].unique()

In [ ]:
df3.head()

In [ ]:
df3.info()

In [ ]:
df3['BHK']=df3['size'].apply(lambda x: int(x.split(' ')[0]))

In [ ]:
df3.head()

In [ ]:
df3['BHK']

In [ ]:
df3['BHK'].unique()

In [ ]:
df3[df3['BHK']==43]

In [ ]:
df3[df3['BHK']>20]

In [ ]:
df3[df3['BHK']<20]

Next Columns

In [ ]:
df3['total_sqft'].unique()

In [ ]:
df3.info()

Make a function that convert the values in sqft to float by try and except method

In [ ]:
def is_float(x):
    try:
         float(x)
    
    except:
        return False
    return True

#this function chk that if we can convert the x into float 
#try method helps to convert the x into float 
#if the value is in string try method help to convert it into the float 
#if the value is not converted into the float it execute in except method or it return the false 



In [ ]:
df3.head()

In [ ]:
#Apply the function
df3[df3['total_sqft'].apply(is_float)]
#it will the true value like float converting value 

In [ ]:
#but i want to see thoes values who's not converted into float so use ~ this is a negate function 
df3[~df3['total_sqft'].apply(is_float)]

In [ ]:
def convert_sqft_to_num(x):
    tokens=x.split('-')
    if len(tokens)==2:
        return (float(tokens[0])+float(tokens[1])/2)
    try:
        return float(x)
    except:
        return None


In [ ]:
convert_sqft_to_num("1521")

In [ ]:
df4=df3.copy()

In [ ]:
df4.head()

In [ ]:
df4['total_sqft']=df4['total_sqft'].apply(convert_sqft_to_num)

In [ ]:
df4.head()

In [ ]:
df4.tail()

In [ ]:
df4.loc[30]

Feature Engineering 

In [ ]:
df5=df4.copy()

In [ ]:
df5.head()

In [ ]:
df5['Price_Per_Sqft']=df5['price']*100000/df5['total_sqft']

In [ ]:
df5.head()

In [ ]:
df5['location'].value_counts()

In [ ]:
df5.groupby('location')['location'].agg('count')

In [ ]:
len(df5['location'])

clean the whitespaces ' jp nagar '

In [ ]:
df5['location']=df5['location'].apply(lambda x: x.strip())

In [ ]:
df5['location'].head()

In [ ]:
len(df5['location'])

In [ ]:
df5['location'].value_counts()

In [ ]:
location_stat=df5['location'].value_counts().sort_index(ascending=False)

In [ ]:
location_stat.head()

In [ ]:
len(location_stat[location_stat<=10])

In [ ]:
location_stat_less_then_10=location_stat[location_stat<=10]

In [ ]:
len(location_stat_less_then_10)

In [ ]:
len(df5['location'].unique())

In [ ]:
df5['location']=df5['location'].apply(lambda x: 'other' if x in location_stat_less_then_10 else x)

In [ ]:
df5['location'].head()

In [ ]:
len(df5['location'].unique())

In [ ]:
df5['location'].unique()

In [ ]:
df5.head()

In [ ]:
df6=df5[~(df5['total_sqft']/df5['BHK']<300)]

In [ ]:
df6.head()

In [ ]:
len(df6['location'])

In [ ]:
def remove_pps_outlier(df):
    df_out=pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m=np.mean(subdf.Price_Per_Sqft)
        st=np.std(subdf.Price_Per_Sqft)
        reduced_df=subdf[(subdf.Price_Per_Sqft>(m-st )) & (subdf.Price_Per_Sqft<=(m+st))]
        df_out=pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out


In [ ]:
df7=remove_pps_outlier(df6)

In [ ]:
df7.head()

In [ ]:
df7.shape

In [ ]:
def remove_bhk_outlier(df):
    exclude_indices=np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats={}
        for bhk, bhk_df in location_df.groupby('BHK'):
            bhk_stats[bhk]={
                'mean':np.mean(bhk_df.Price_Per_Sqft),
                'std':np.std(bhk_df.Price_Per_Sqft),
                'count':bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('BHK'):
            stats=bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices=np.append(exclude_indices,bhk_df[bhk_df.Price_Per_Sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
df8=remove_bhk_outlier(df7)
df8.shape


In [ ]:
def plot_scatter_chart(df,location):
    bhk2=df[(df.location==location) & (df.BHK==2)]
    bhk3=df[(df.location==location) & (df.BHK==3)]
    plt.scatter(bhk2.total_sqft,bhk2.price,color='red',label='2 BHK',s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,label='3 BHK',color="green",s=50,marker='+')
    plt.xlabel('total_sqft_area')
    plt.ylabel('price')
    plt.title(location)
plot_scatter_chart(df8,'Hebbal')


In [ ]:
plt.hist(df7['Price_Per_Sqft'],rwidth=0.8)

In [ ]:
df8['bath'].unique()

In [ ]:
plt.hist(df8['bath'])

In [ ]:
df9=df8.drop(['Price_Per_Sqft','size'],axis=1)

In [ ]:
df9

In [ ]:
df9.info()

In [ ]:
dummies=pd.get_dummies(df9['location'])

In [ ]:
df10=pd.concat([df9,dummies.drop('other',axis=1)],axis=1)

In [ ]:
df10.head()

In [ ]:
df11=df10.drop('location',axis=1)

In [ ]:
df11.head()

In [ ]:
x=df11.drop('price',axis=1)

In [ ]:
x

In [ ]:
x=x.astype(int)

In [ ]:
x

In [ ]:
y=df11['price']

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=10)

In [ ]:
len(x_train)

In [ ]:
len(x_test)

In [ ]:
print(x.dtypes)

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_train,y_train)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=10)
cross_val_score(LinearRegression(),x,y,cv=cv)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
def find_best_model_using_gridsearchcv(x,y):
    algos={
        'LinearRegression':{
           'model':LinearRegression(),
           'params':{
               #'normalize':[True,False]
           }
        },
        'lasso':{
            'model':Lasso(),
            'params':{
                'alpha':[1,2],
                'selection':['random','cyclic']
            }
        },
        'decision_tree':{
            'model':DecisionTreeRegressor(),
            'params':{
                'criterion':['squared_error','friedman_mse'],
                'splitter':['best','random']
            }
        }
    }
    
    score=[]
    cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=10)
    for algos_name,config in algos.items():
        gs=GridSearchCV(config['model'],config['params'],cv=cv,return_train_score=False)
        gs.fit(x,y)
        score.append({
            'model':algos_name,
            'best_score':gs.best_score_,
            'best_params':gs.best_params_
        })

    return pd.DataFrame(score,columns=['model','best_score','best_params'])
    
    
find_best_model_using_gridsearchcv(x,y)

In [ ]:
x.columns

In [ ]:

def predict_price(location,sqft,bath,BHK,x,lr):
    loc_index=np.where(x.columns==location)[0][0]

    x=np.zeros(len(x.columns))
    x[0]=sqft
    x[1]=bath
    x[2]=BHK
    if loc_index>=0:
        x[loc_index]=1
    return lr.predict([x])[0]

In [ ]:
predict_price('1st Phase JP Nagar',1000,2,2,x,lr)

In [ ]:
import pickle
with open('project.pickle','wb') as f:
    pickle.dump(lr,f)